In [2]:
import os
import json
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path


class Imagej_UI:

    """
    A class to display a UI of cookiecutter template file fields.
    """

    def __init__(self, path=None):
        self.wd = path

        if path is None:
            self.wd = Path(os.path.abspath(''))

        # Recursively search working directory for cookietin directory
        self.cookietin = [path for path in self.wd.rglob('*') if path.name == 'cookietin']

        self.plugins = []
        self.names = []
        self.layout = widgets.Layout(width='auto', height='40px')
        self.style = {'description_width': 'initial'}

        # Get all the cookie cutter template file paths and plugin names
        for path in self.cookietin:
            self.plugins += [plugin for plugin in self.wd.rglob('*') if plugin.name == 'cookiecutter.json']
            self.names += [plugin.parent.name for plugin in self.plugins]
            self.d = {name: path for name, path in zip(self.names, self.plugins)}

        # Sort plugin names
        self.names.sort()

        # Create dropdown to select plugin template file
        self.p = widgets.Dropdown(
            options=self.names,
            value=self.names[0],
            description='Select Plugin:',
            disabled=False,
        )

        # Get initial data from first plugin in list
        with open(self.plugins[0], 'r') as fhand:
            self.data = json.load(fhand)
            fhand.close()

    def get_fields(self, dic, key=''):
        """
        A method to get all fields and values from cookiecutter template file.
        """
        data = {}

        # Recursively search the dictionary for field values
        for k in dic.keys():
            if isinstance(dic[k], dict):
                data.update(self.get_fields(dic[k], key+':'+k))

            elif isinstance(dic[k], str):
                data[str(key+':'+k)[1:]] = str(dic[k])

            elif isinstance(dic[k], list):
                data[str(key+':'+k)[1:]] = str(dic[k]).replace("'", "").replace('"', '')

        return data

    def show_fields(self, path):
        """
        A method to create widgets using data from selected cookiecutter
        template file.
        """

        with open(self.d[self.p.value], 'r') as fhand:
            self.data = json.load(fhand)
            fhand.close()

        data = self.get_fields(self.data)
        self.w = []

        fields = [f for f in data.keys()]
        for f in fields:
            self.w += [widgets.Text(value=data[f], description=f, layout=self.layout, style=self.style)]

        self.save = widgets.Button(
            value=False,
            description='Save File',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Save to {}'.format(self.d[path]),
            icon='save'  # (FontAwesome names without the `fa-` prefix)
        )

        self.generate = widgets.Button(
            value=False,
            description='Generate Plugin',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Generate plugin with {}'.format(self.d[path]),
            icon='check'  # (FontAwesome names without the `fa-` prefix)
        )

        self.containerize = widgets.Button(
            value=False,
            description='Build Docker',
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Build docker image and push to repo',
            icon='check'  # (FontAwesome names without the `fa-` prefix)
        )

        self.save.on_click(self.on_save_clicked)
        self.generate.on_click(self.on_generate_clicked)
        self.containerize.on_click(self.on_containerize_clicked)
        self.output = widgets.Output()
        display(*self.w, self.save, self.generate, self.containerize, self.output)

    def update_data(self, d, key_list, new_value):
        """
        A method to update template data with user's input.
        """

        k = key_list[0]

        if len(key_list) == 1:
            if new_value[0] == '[' and new_value[-1] == ']':
                # Remove first and last parenthesis and convert to list
                d[k] = new_value[1:-1].split(',')
                # Strip whitespace from values in list
                d[k] = [value.strip() for value in d[k]]
            else:
                d[k] = new_value

        else:
            d[key_list[0]] = self.update_data(d[k], key_list[1:], new_value)

        return d

    def on_save_clicked(self, b):
        """
        A method to save the updated cookiecutter template file.
        """

        with self.output:
            print("Saved to {}".format(self.d[self.p.value]))

            for w in self.w:
                k = w.description.split(':')
                v = w.value
                self.data = self.update_data(self.data, k, v)

            with open(self.d[self.p.value], 'w') as fhand:
                json.dump(self.data, fhand, indent=4)
                fhand.close()

    def on_generate_clicked(self, b):
        """
        A method to genetate the plugin from the cookiecutter template file.
        """
        with self.output:
            print('Generating the {} plugin'.format(self.p.value))
            os.system('python generate.py --plugins {}\n'.format(self.p.value))

    def on_containerize_clicked(self, b):
        """
        A method to build and push the plugin's docker container.
        """
        with self.output:

            # Define the plugin directory
            plugin_dir = self.wd.parents[1].joinpath('polus-imagej-{}-plugin'.format(self.p.value.lower()))

            # Define the temp destination of plugin directory
            temp_dir = '~/shared/wipp/temp/plugins'

            # Copy plugin to temp/plugins directory
            os.system('cp -r {} {}'.format(plugin_dir, temp_dir))

            plugin_name = self.p.value.lower() + '-plugin' 
            docker_hub_repo = 'imagej-{}-plugin'.format(self.p.value.lower())
            version = self.data['version']

            print('Building Docker Container for {}'.format(plugin_name))

            # Run kaniko
            os.system('python imagej_kaniko.py --plugin_name {} --docker_hub_repo {} --version {}' \
                      .format(plugin_name, docker_hub_repo, version))

    def display_ui(self):
        """
        A method to start and display the interactive dashboard.
        """

        self.ui = widgets.interactive(self.show_fields, path=self.p)
        display(self.ui)


# Define the dashboard
ui = Imagej_UI()
# Start and display dashboard
ui.display_ui()

interactive(children=(Dropdown(description='Select Plugin:', options=('convert-bit', 'convert-cfloat32', 'conv…